In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

# add the path to my packages to system paths so they can be imported
import sys
sys.path.append('/home/yasamanparhizkar/Documents/yorku/01_thesis/simgraph/code/my_packages')
# sys.path.append('F:\MAScThesis\code\my_packages')
# sys.path.append('/home/yasamanparhizkar/Documents/thesis/code/my_packages')

import dataprocess.data_handler_03 as dh
import assessment.assess_simgraph_02 as asg
import simgraph.my_simgraph as sg

In [2]:
# import importlib
# importlib.reload(sg)

# Load and group neuron spike data

In [3]:
# load all spike data from file
spikes_dp = '../../../local_data/original_files/'
grouped_data = np.load(spikes_dp+'summed_spikes.npy') # you can load alternative files from the same directory

I_order_10 = [54, 35, 10, 60, 74, 9, 61, 56, 91, 104]
grouped_data.shape

(297, 1141, 1)

# Check the dataset

In [4]:
def get_mnist_labels(data_params):
    return np.loadtxt(data_params['features_dp']+'lbls.csv')

def transform_mnistsift(fv):
    return fv[::4]

def transform_slowfast(fv):
    """
    Transform to be applied on feature vectors.
    
    Input: fv
    fv - 1xDf torch tensor representing a feature vector
    
    Output: fvv
    fvv - 1xDf' torch tensor representing the transformed feature vector
    """
    
    # for faster run and less memory usage
    fvv = fv[::200]
    
    # for numerical stability during GD
    # fvv = fvv * 10
    
    return fvv

def transform_sift3d(fv):
    return fv[::10]*10000

def transform_soenet(fv):
    return fv[::5]

# data retrieval params
# data_params = {'func': dh.datapoint_sift, 'lbl_func': get_mnist_labels, 'features_dp': '../../data/fe_exp/mnist-sift/', \
#                'spike_data': None, 'group_id': None, 'transform': transform_mnistsift, 'ind_min': 0, 'ind_max': 13203, 'feature_id':'mnist-sift'}

# data_params = {'func': dh.datapoint_numpy, 'lbl_func': dh.get_labels, 'features_dp': '../../data/features/slowfast/slowfast_4732_numpy/', \
#                'spike_data': grouped_data, 'group_id': 0, 'transform': transform_slowfast, 'ind_min': 1*1141+0, 'ind_max': 2*1141-1, 'feature_id':'slowfast'}

data_params = {'func': dh.datapoint_numpy, 'lbl_func': dh.get_labels, 'features_dp': '../../data/features/sift3d/fvs_s1_with_kp/desc/', \
               'spike_data': grouped_data, 'group_id': 0, 'transform': transform_sift3d, 'ind_min': 1*1141+0, 'ind_max': 2*1141-1, 'feature_id':'sift3d'}

# data_params = {'func': dh.datapoint_numpy, 'lbl_func': dh.get_labels, 'features_dp': '../../data/features/soenet/soenet3/features_2layer/', \
#                'spike_data': grouped_data, 'group_id': 0, 'transform': transform_soenet, 'ind_min': 1*1141+41, 'ind_max': 2*1141-1, 'feature_id':'soenet'}

In [5]:
train_num = 200
val_num = 50

train_num, val_num, train_data, val_data = \
dh.random_train_val(train_num, val_num, data_params, seed=0)

# optional normalization
# train_data['des'] = dh.normalize(train_data['des'])
# val_data['des'] = dh.normalize(val_data['des'])

# show statistics
print('feature_id: ', data_params['feature_id'])
print('train_num = ', train_num, ', val_num = ', val_num)
print('number of features: ', train_data['des'].shape[1])

print('train_smpls = ', train_data['smpls'], '\nval_smpls = ', val_data['smpls'])
print('train_lbls = ', train_data['lbls'], '\nval_lbls = ', val_data['lbls'])
print('train_des = ', train_data['des'], '\nval_des = ', val_data['des'])
print('minimum value of train features: ', np.min(train_data['des']))
print('mean value of train features: ', np.mean(train_data['des']))
print('median value of train features: ', np.median(train_data['des']))
print('maximum value of train features: ', np.max(train_data['des']))

feature_id:  sift3d
train_num =  200 , val_num =  50
number of features:  77
train_smpls =  [1907 2108 1270 1373 1243 1977 2132 1565 1223 1756 1851 1488 2001 1789
 1596 1419 1344 1534 1548 1781 1502 1720 1257 1717 2110 1342 2171 1199
 2142 1665 1597 1775 1642 1794 1215 1395 1635 1445 1282 1832 1690 1224
 1844 1965 1942 1580 1915 1748 1487 1455 1148 1216 1477 1500 2213 1824
 2233 1715 2264 2135 1634 1294 2175 1588 1866 1702 1291 1308 2127 2038
 1973 1918 1609 2136 1416 2102 1813 1886 1345 1927 2016 1391 1584 1145
 2268 1155 1454 1379 1172 2019 2204 2182 1966 1305 1512 2156 1301 1656
 1303 1188 1759 2239 2223 2046 1570 1791 1899 1506 1812 1382 2140 2043
 2201 1161 2009 1732 1797 1400 2078 2196 2237 2118 1876 1540 1593 2259
 1302 1249 2155 1220 1364 2215 1525 2186 1992 2177 1939 2169 2203 1471
 1716 1190 1192 2252 1796 1944 1514 1661 2044 1516 1440 1167 1171 1873
 2080 2041 1773 1463 1902 1922 1388 1649 2100 1990 1930 1699 1769 1700
 2209 1559 2273 2072 2090 2126 2211 2133 1263 1211 1436 

# Assess the model's performance with random tests

In [6]:
# import importlib
# importlib.reload(asg)

In [7]:
# graph construction parameters (for both lmnn and obj1)
sg_params = {'Dt': None, 'Dv': None, 'Dvt': None, 'mu': 20,\
             'cnstr_method_tt': 'time-inc-ord', 'cnstr_method_vv': 'time-inc-ord', 'cnstr_method_vt': 'time',\
             'train_t': None, 'val_t': None, \
             'edges_tt':None, 'edges_vv':None, 'edges_vt':None, }

# gradient descent parameters (only used for factobj1)
gd_opt_params = { 'epsilon0':1, 'epsilon_decay':0.5, 'epsilon_jump': 1.7, \
                'num_its': 500, 'check_freq':501, 'print_checks':False, 'Theta0':None, \
                'force_all_its': True, 'threshold': -1}

# randomization parameters
rnd_params = {'train_sizes': [50, 200, 400], 'val_sizes': [100], 'train_its': 100, 'val_its': 10, 'seed': None}

# parameters to visualize the optimized M
f_sz = train_data['des'].shape[1] # must match data_params
xloc = np.broadcast_to(np.arange(f_sz), (f_sz, f_sz))
yloc = xloc.T
fig_params = {'rmark_th': 50, 'f_sz': f_sz, 'xloc': xloc, 'yloc': yloc}

# path to save the results
res_path1 = '../../../local_data/experiments/sift3d/sift3d_sg/temp1/'

In [8]:
val_num_res, val_num_err = asg.assess_sg_model(data_params, sg_params, gd_opt_params, rnd_params, fig_params, res_path1)

Done. Elapsed time: 119113.766 sec


In [9]:
asg.plot_curves(rnd_params, sg_params, res_path1)